### Setup libs

In [1]:
import json
from datetime import datetime, timedelta
import pandas as pd
from os import listdir
from os.path import isfile, join

### Reading multiple json raw files (data source)

In [2]:
def concatened_json_files(path):
    directory_path = path
    concatenated_file = []
    file_counter = 0

    file_list = [f for f in listdir(directory_path) if isfile(join(directory_path, f))]

    for file in file_list:
        file_counter+=1
        with open(f'{directory_path}{file}') as f:
            for obj in json.load(f):
                concatenated_file.append(obj)
    
    print(f'Concatened {file_counter} json files into one.')
    return concatenated_file

In [3]:
df = pd.json_normalize(concatened_json_files('../datasets/anime/raw_data/'))

Concatened 517 json files into one.


In [4]:
df.head(5)

,id,title,photo,synopsis,background,data_extraction,links.episodes,links.stats,links.characters & staff,details.Synonyms,...,details.Duration,details.Rating,details.Score,details.ScoredBy,details.Ranked,details.Popularity,details.Members,details.Favorites,details.English,links
0,6604,Kuruneko,https://cdn.myanimelist.net/images/anime/6/205...,Based on a manga about cats and their sake lov...,No background information,2021-02-16 17:48:51,https://myanimelist.net/anime/6604/Kuruneko/ep...,https://myanimelist.net/anime/6604/Kuruneko/stats,https://myanimelist.net/anime/6604/Kuruneko/ch...,Kuru Neko,...,2 min. per ep.,G - All Ages,7.31,706,#2308,#7070,"4,030",9,NaN,NaN
1,6607,Superflat First Love,https://cdn.myanimelist.net/images/anime/7/222...,6 years after she was swallowed by the Panda G...,No background information,2021-02-16 17:48:54,NaN,https://myanimelist.net/anime/6607/Superflat_F...,https://myanimelist.net/anime/6607/Superflat_F...,NaN,...,3 min.,G - All Ages,5.67,1945,#9226,#7644,"3,212",1,NaN,NaN
2,6609,Pucca,https://cdn.myanimelist.net/images/anime/3/151...,Pucca is the young daughter of a Chinese noodl...,No background information,2021-02-16 17:48:55,https://myanimelist.net/anime/6609/Pucca/episode,https://myanimelist.net/anime/6609/Pucca/stats,https://myanimelist.net/anime/6609/Pucca/chara...,Jjajang Sonyeo Ppukka,...,23 min. per ep.,G - All Ages,6.56,5996,#5582,#4673,"11,369",17,NaN,NaN
3,6610,Ibara no Ou,https://cdn.myanimelist.net/images/anime/9/870...,"""Medusa,"" a deadly virus becomes a worldwide e...",No background information,2021-02-16 17:48:57,NaN,https://myanimelist.net/anime/6610/Ibara_no_Ou...,https://myanimelist.net/anime/6610/Ibara_no_Ou...,Ibara no Oh,...,1 hr. 50 min.,R+ - Mild Nudity,7.00,29620,#3634,#1995,"60,159",156,King of Thorn,NaN
4,6616,Watashi no Coffee Samurai: Jihanki-teki na Kar...,https://cdn.myanimelist.net/images/anime/8/354...,"After falling in battle, the samurai Jin wishe...",No background information,2021-02-16 17:49:03,NaN,https://myanimelist.net/anime/6616/Watashi_no_...,https://myanimelist.net/anime/6616/Watashi_no_...,My Coffee Samurai: A Vending Machine Boyfriend,...,30 min.,G - All Ages,6.05,1142,#8013,#8073,"2,697",7,Coffee Samurai,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21755 entries, 0 to 21754
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        21755 non-null  int64  
 1   title                     21755 non-null  object 
 2   photo                     17758 non-null  object 
 3   synopsis                  21755 non-null  object 
 4   background                21755 non-null  object 
 5   data_extraction           21755 non-null  object 
 6   links.episodes            12194 non-null  object 
 7   links.stats               21754 non-null  object 
 8   links.characters & staff  21754 non-null  object 
 9   details.Synonyms          10950 non-null  object 
 10  details.Japanese          21560 non-null  object 
 11  details.Type              20036 non-null  object 
 12  details.Episodes          21755 non-null  object 
 13  details.Status            21755 non-null  object 
 14  detail

### Number of Rows

In [6]:
print(len(df.index))

21755


### Transforming Ranked and Popularity into numbers 

In [7]:
df = pd.json_normalize(concatened_json_files('../datasets/anime/raw_data/'))
columns_for_treatment = ['Score','Ranked','Popularity','ScoredBy','Members','Favorites']

for column in columns_for_treatment:
    def transform_into_number(df_row):
        if df_row[f'details.{column}'] == 'N/A' or df_row[f'details.{column}'] == '':
            return 0
        if '.' in df_row[f'details.{column}']:
            return float(df_row[f'details.{column}'])
        return int(df_row[f'details.{column}'].replace('#','').replace(',',''))
    df[f'details.{column}'] = df.apply(transform_into_number,axis=1)

df.head(5)

Concatened 517 json files into one.


,id,title,photo,synopsis,background,data_extraction,links.episodes,links.stats,links.characters & staff,details.Synonyms,...,details.Duration,details.Rating,details.Score,details.ScoredBy,details.Ranked,details.Popularity,details.Members,details.Favorites,details.English,links
0,6604,Kuruneko,https://cdn.myanimelist.net/images/anime/6/205...,Based on a manga about cats and their sake lov...,No background information,2021-02-16 17:48:51,https://myanimelist.net/anime/6604/Kuruneko/ep...,https://myanimelist.net/anime/6604/Kuruneko/stats,https://myanimelist.net/anime/6604/Kuruneko/ch...,Kuru Neko,...,2 min. per ep.,G - All Ages,7.31,706,2308,7070,4030,9,NaN,NaN
1,6607,Superflat First Love,https://cdn.myanimelist.net/images/anime/7/222...,6 years after she was swallowed by the Panda G...,No background information,2021-02-16 17:48:54,NaN,https://myanimelist.net/anime/6607/Superflat_F...,https://myanimelist.net/anime/6607/Superflat_F...,NaN,...,3 min.,G - All Ages,5.67,1945,9226,7644,3212,1,NaN,NaN
2,6609,Pucca,https://cdn.myanimelist.net/images/anime/3/151...,Pucca is the young daughter of a Chinese noodl...,No background information,2021-02-16 17:48:55,https://myanimelist.net/anime/6609/Pucca/episode,https://myanimelist.net/anime/6609/Pucca/stats,https://myanimelist.net/anime/6609/Pucca/chara...,Jjajang Sonyeo Ppukka,...,23 min. per ep.,G - All Ages,6.56,5996,5582,4673,11369,17,NaN,NaN
3,6610,Ibara no Ou,https://cdn.myanimelist.net/images/anime/9/870...,"""Medusa,"" a deadly virus becomes a worldwide e...",No background information,2021-02-16 17:48:57,NaN,https://myanimelist.net/anime/6610/Ibara_no_Ou...,https://myanimelist.net/anime/6610/Ibara_no_Ou...,Ibara no Oh,...,1 hr. 50 min.,R+ - Mild Nudity,7.00,29620,3634,1995,60159,156,King of Thorn,NaN
4,6616,Watashi no Coffee Samurai: Jihanki-teki na Kar...,https://cdn.myanimelist.net/images/anime/8/354...,"After falling in battle, the samurai Jin wishe...",No background information,2021-02-16 17:49:03,NaN,https://myanimelist.net/anime/6616/Watashi_no_...,https://myanimelist.net/anime/6616/Watashi_no_...,My Coffee Samurai: A Vending Machine Boyfriend,...,30 min.,G - All Ages,6.05,1142,8013,8073,2697,7,Coffee Samurai,NaN


### Removing dirt data from Producers, Licensors and Studios

In [8]:
columns_for_treatment = ['Producers','Licensors','Studios']

for column in columns_for_treatment:
    def treat_blank_values(df_row):
        if 'add some' in df_row[f'details.{column}'] or 'None found' in df_row[f'details.{column}']:
            return None
        else:
            return df_row[f'details.{column}']
    df[f'details.{column}'] = df.apply(treat_blank_values,axis=1)


df.head(5)

,id,title,photo,synopsis,background,data_extraction,links.episodes,links.stats,links.characters & staff,details.Synonyms,...,details.Duration,details.Rating,details.Score,details.ScoredBy,details.Ranked,details.Popularity,details.Members,details.Favorites,details.English,links
0,6604,Kuruneko,https://cdn.myanimelist.net/images/anime/6/205...,Based on a manga about cats and their sake lov...,No background information,2021-02-16 17:48:51,https://myanimelist.net/anime/6604/Kuruneko/ep...,https://myanimelist.net/anime/6604/Kuruneko/stats,https://myanimelist.net/anime/6604/Kuruneko/ch...,Kuru Neko,...,2 min. per ep.,G - All Ages,7.31,706,2308,7070,4030,9,NaN,NaN
1,6607,Superflat First Love,https://cdn.myanimelist.net/images/anime/7/222...,6 years after she was swallowed by the Panda G...,No background information,2021-02-16 17:48:54,NaN,https://myanimelist.net/anime/6607/Superflat_F...,https://myanimelist.net/anime/6607/Superflat_F...,NaN,...,3 min.,G - All Ages,5.67,1945,9226,7644,3212,1,NaN,NaN
2,6609,Pucca,https://cdn.myanimelist.net/images/anime/3/151...,Pucca is the young daughter of a Chinese noodl...,No background information,2021-02-16 17:48:55,https://myanimelist.net/anime/6609/Pucca/episode,https://myanimelist.net/anime/6609/Pucca/stats,https://myanimelist.net/anime/6609/Pucca/chara...,Jjajang Sonyeo Ppukka,...,23 min. per ep.,G - All Ages,6.56,5996,5582,4673,11369,17,NaN,NaN
3,6610,Ibara no Ou,https://cdn.myanimelist.net/images/anime/9/870...,"""Medusa,"" a deadly virus becomes a worldwide e...",No background information,2021-02-16 17:48:57,NaN,https://myanimelist.net/anime/6610/Ibara_no_Ou...,https://myanimelist.net/anime/6610/Ibara_no_Ou...,Ibara no Oh,...,1 hr. 50 min.,R+ - Mild Nudity,7.00,29620,3634,1995,60159,156,King of Thorn,NaN
4,6616,Watashi no Coffee Samurai: Jihanki-teki na Kar...,https://cdn.myanimelist.net/images/anime/8/354...,"After falling in battle, the samurai Jin wishe...",No background information,2021-02-16 17:49:03,NaN,https://myanimelist.net/anime/6616/Watashi_no_...,https://myanimelist.net/anime/6616/Watashi_no_...,My Coffee Samurai: A Vending Machine Boyfriend,...,30 min.,G - All Ages,6.05,1142,8013,8073,2697,7,Coffee Samurai,NaN


### Dump dataframe as csv for sample analysis

In [9]:
df.to_csv('../datasets/anime/sandbox/anime_dataset.csv',sep='|',index=False)

### Bleach preparation

In [10]:
pd.set_option('display.max_columns', None)
bleach = df[df['id'] == 269]
prep_bleach = bleach[['id','title','details.Episodes','details.Duration','details.Status','details.Aired']]
prep_bleach.head(1)

,id,title,details.Episodes,details.Duration,details.Status,details.Aired
2430,269,Bleach,366,24 min. per ep.,Finished Airing,"Oct 5, 2004 to Mar 27, 2012"


### Dates preparation using column 'Aired' and 'Duration' as source

In [17]:
prep_bleach_aired = prep_bleach


def convert_date_format(date):
    try:
        date_transformed = datetime.strptime(date, "%b %d, %Y")
        return datetime.strftime(date_transformed, '%Y-%m-%d')
    except:
        return f'Failed: {date}'

    
def get_aired_from_date(df_row):
    if df_row['details.Aired'] == 'Not available':
        return None
    if len(df_row['details.Aired']) == 4:
        return convert_date_format(f'Jan 1, {df_row["details.Aired"]}')
    if df_row['details.Status'] == 'Not yet aired':
        return None
    else:
        aired = df_row['details.Aired'][:12].rstrip()
        
        if 'to ?' in aired and len(aired) == 9:
            return convert_date_format(f'Jan 1, {aired[:4]}')
        
        if 'to' not in aired and len(aired) == 9:
            partial_date = aired.rstrip().lstrip()
            return convert_date_format(f'{partial_date[:3]} 1, {partial_date[5:]}')
        
        elif 'to' in aired:
            if len(aired) == 9:
                partial_date = aired.replace('to','').rstrip().lstrip()
                return convert_date_format(f'{partial_date[:3]} 1, {partial_date[5:]}')
            if len(aired[:aired.find('to')-1].rstrip()) == 4:
                return convert_date_format(f'Jan 1, {aired[:aired.find("to")-1].rstrip()}')
            
        else:
            return convert_date_format(aired)


def get_aired_to_date(df_row):
    if df_row['details.Aired'] == 'Not available':
        return None
    if len(df_row['details.Aired']) == 4:
        return convert_date_format(f'Jan 1, {df_row["details.Aired"]}')
    if 'to ?' in df_row['details.Aired']:
        return None

    else:
        aired = df_row['details.Aired'][-12:].lstrip()

        if len(aired) == 9:
            partial_date = aired.replace('to','').rstrip().lstrip()
            return convert_date_format(f'{partial_date[:3]} 1, {partial_date[5:]}')

        if 'to' in aired:
            if len(aired[aired.find('to')+3:]) == 4:
                return convert_date_format(f'Dec 31, {aired[aired.find("to")+3:]}')
            if len(aired) == 9:
                partial_date = aired.replace('to','').rstrip().lstrip()
                return convert_date_format(f'{partial_date[:3]} 1, {partial_date[5:]}')

        else:
            return convert_date_format(aired)

    
def calculate_airing_duration(df_row):
    if df_row['details.Aired'] == 'Not available':
        return None
    
    from_date = df_row['aired_from']
    to_date = df_row['aired_to']
    
    if to_date is None and df_row['details.Status'] == 'Currently Airing':
        to_date = datetime.now().strftime("%Y-%m-%d")
    
    try:
        from_datetime = datetime.strptime(from_date, "%Y-%m-%d")
        to_datetime = datetime.strptime(to_date, "%Y-%m-%d")
    except:
        return 0
    else:
        return int((to_datetime - from_datetime).total_seconds() / 86400)


def get_hours_per_episode(df_row):
    if 'hr' in df_row['details.Duration']:
        return int(df_row['details.Duration'][:df_row['details.Duration'].find('hr')-1])
    else:
        return 0
    

def get_minutes_per_episode(df_row):
    if 'min' in df_row['details.Duration']:
        if 'hr.' in df_row['details.Duration']:
            return int(df_row['details.Duration'][df_row['details.Duration'].find('hr.')+4:df_row['details.Duration'].find('min')-1])
        else:
            return int(df_row['details.Duration'][:df_row['details.Duration'].find('min')-1])
    else:
        return 0

    
def get_seconds_per_episode(df_row):
    if 'sec' in df_row['details.Duration']:
        return int(df_row['details.Duration'][:df_row['details.Duration'].find('sec')-1])
    else:
        return 0
    
    
def calculate_episode_lenght_in_seconds(df_row):
    return df_row['hours_per_episode']*3600 + df_row['minutes_per_episode']*60 + df_row['seconds_per_episode']
    

def calculate_airing_duration_in_seconds(df_row):
    if df_row['details.Aired'] == 'Not available':
        return None
    
    if df_row['details.Episodes'] != 'Unknown':
        return int(df_row['details.Episodes']) * df_row['episode_lenght_in_seconds']
    else:
        return None
    
def fix_episodes(df_row): 
    if df_row['details.Episodes'] != 'Unknown':
        return int(df_row['details.Episodes'])
    else:
        return None
    
prep_bleach_aired['aired_from'] = prep_bleach_aired.apply(get_aired_from_date,axis=1)
prep_bleach_aired['aired_to'] = prep_bleach_aired.apply(get_aired_to_date,axis=1)
prep_bleach_aired['days_in_air'] = prep_bleach_aired.apply(calculate_airing_duration,axis=1)
prep_bleach_aired['hours_per_episode'] = prep_bleach_aired.apply(get_hours_per_episode,axis=1)
prep_bleach_aired['minutes_per_episode'] = prep_bleach_aired.apply(get_minutes_per_episode,axis=1)
prep_bleach_aired['seconds_per_episode'] = prep_bleach_aired.apply(get_seconds_per_episode,axis=1)
prep_bleach_aired['episode_lenght_in_seconds'] = prep_bleach_aired.apply(calculate_episode_lenght_in_seconds,axis=1)
prep_bleach_aired['seconds_in_air'] = prep_bleach_aired.apply(calculate_airing_duration_in_seconds,axis=1)
prep_bleach_aired['details.Episodes'] = prep_bleach_aired.apply(fix_episodes,axis=1)

prep_bleach_aired

<ipython-input-17-5f58f8e8b24e>:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prep_bleach_aired['aired_from'] = prep_bleach_aired.apply(get_aired_from_date,axis=1)
<ipython-input-17-5f58f8e8b24e>:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prep_bleach_aired['aired_to'] = prep_bleach_aired.apply(get_aired_to_date,axis=1)
<ipython-input-17-5f58f8e8b24e>:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

,id,title,details.Episodes,details.Duration,details.Status,details.Aired,aired_from,aired_to,days_in_air,hours_per_episode,minutes_per_episode,seconds_per_episode,episode_lenght_in_seconds,seconds_in_air
2430,269,Bleach,366,24 min. per ep.,Finished Airing,"Oct 5, 2004 to Mar 27, 2012",2004-10-05,2012-03-27,2730,0,24,0,1440,527040


### Analyzing Genres

### Genres preparation using genres_list

In [12]:
df = pd.json_normalize(concatened_json_files('../datasets/anime/raw_data/'))

Concatened 517 json files into one.


In [13]:
def parse_list_in_column(data_dict,column_for_parse):
    result_list = []
    for obj in data_dict:
        for value in obj['details'][column_for_parse]:
            result_list.append(value)

    return list(set(result_list))

In [14]:
animes_dict = concatened_json_files('../datasets/anime/raw_data/')
genre_df = df[['id','details.Genres']]

for genre in parse_list_in_column(animes_dict,'Genres'):
    def create_genre_column(df_row):
        if genre in df_row['details.Genres']:
            return 1
        else:
            return 0
    genre_df[genre] = genre_df.apply(create_genre_column,axis=1)

genre_df.to_parquet(path='../datasets/anime/enhanced_data/animes_genres.parquet')
print(len(genre_df.index))

Concatened 517 json files into one.


<ipython-input-14-e6329522df5c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genre_df[genre] = genre_df.apply(create_genre_column,axis=1)


21755


In [15]:
animes_genres = pd.read_parquet(path='../datasets/anime/enhanced_data/animes_genres.parquet')
animes_genres.tail(5)

,id,details.Genres,Slice of Life,Yaoi,Game,Magic,Comedy,Kids,Psychological,Shounen Ai,Seinen,Dementia,Drama,Space,Cars,Mecha,Super Power,Shoujo,Shoujo Ai,Thriller,Yuri,Martial Arts,Supernatural,Action,Ecchi,Vampire,Fantasy,Military,Harem,Shounen,Adventure,School,Police,Historical,Josei,Hentai,Parody,Sports,Mystery,Horror,Sci-Fi,Samurai,Romance,Music,Demons
21750,7592,"[Action, Demons, Shounen, Supernatural]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
21751,7593,"[Harem, Comedy, Romance, Ecchi, School, Seinen]",0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
21752,7596,"[Comedy, Parody, Sci-Fi]",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
21753,7597,"[Comedy, Seinen, Slice of Life]",1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21754,7598,"[Sci-Fi, Mystery, Thriller]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0


In [16]:
animes_genres = pd.read_parquet(path='../datasets/anime/enhanced_data/prepared_animes.parquet')
animes_genres.tail(5)

,id,title,photo,synopsis,background,data_extraction,links.episodes,links.stats,links.characters & staff,details.Synonyms,details.Japanese,details.Type,details.Episodes,details.Status,details.Aired,details.Premiered,details.Broadcast,details.Producers,details.Licensors,details.Studios,details.Source,details.Genres,details.Duration,details.Rating,details.Score,details.ScoredBy,details.Ranked,details.Popularity,details.Members,details.Favorites,details.English,links,aired_from,aired_to,days_in_air,hours_per_episode,minutes_per_episode,seconds_per_episode,episode_lenght_in_seconds,seconds_in_air
21750,7592,Nurarihyon no Mago,https://cdn.myanimelist.net/images/anime/9/756...,"Rikuo Nura, a part-youkai and part-human boy, ...",Advanced screenings of the first two episodes ...,2021-02-16 18:18:12,https://myanimelist.net/anime/7592/Nurarihyon_...,https://myanimelist.net/anime/7592/Nurarihyon_...,https://myanimelist.net/anime/7592/Nurarihyon_...,"The Grandson of Nurarihyon, Grandchild of Nura...",ぬらりひょんの孫,TV,24,Finished Airing,"Jul 6, 2010 to Dec 21, 2010",Summer 2010,Tuesdays at 01:44 (JST),"[Yomiuri Telecasting, Yomiko Advertising, Pony...",[VIZ Media],[Studio Deen],Manga,"[Action, Demons, Shounen, Supernatural]",24 min. per ep.,PG-13 - Teens 13 or older,7.65,96208,1144,657,222495,1341,Nura: Rise of the Yokai Clan,NaN,2010-07-06,2010-12-21,168.0,0,24,0,1440,34560.0
21751,7593,Kiss x Sis (TV),https://cdn.myanimelist.net/images/anime/3/255...,"After Keita Suminoe's mother passed away, his ...",Episode 1 was pre-streamed on Bandai Channel o...,2021-02-16 18:18:13,https://myanimelist.net/anime/7593/Kiss_x_Sis_...,https://myanimelist.net/anime/7593/Kiss_x_Sis_...,https://myanimelist.net/anime/7593/Kiss_x_Sis_...,"Kiss x Sis (2010), Kissxsis",キスシス,TV,12,Finished Airing,"Apr 5, 2010 to Jun 21, 2010",Spring 2010,Unknown,[Starchild Records],None,[feel.],Manga,"[Harem, Comedy, Romance, Ecchi, School, Seinen]",24 min. per ep.,R+ - Mild Nudity,6.71,254975,4865,295,438390,1549,None,NaN,2010-04-05,2010-06-21,77.0,0,24,0,1440,17280.0
21752,7596,Haiyoru! Nyaruani,https://cdn.myanimelist.net/images/anime/5/223...,Animated Flash shorts based on the light novel...,No background information,2021-02-16 18:18:16,https://myanimelist.net/anime/7596/Haiyoru_Nya...,https://myanimelist.net/anime/7596/Haiyoru_Nya...,https://myanimelist.net/anime/7596/Haiyoru_Nya...,"Haiyore! Nyaruko-san, Haiyoru! Nyarl-ani",這いよる!ニャルアニ,OVA,9,Finished Airing,"Mar 17, 2010",None,None,[SoftBank Creative],None,[DLE],Light novel,"[Comedy, Parody, Sci-Fi]",2 min. per ep.,PG-13 - Teens 13 or older,6.29,10823,6951,3182,26532,25,Haiyoru! Nyaruani,NaN,2010-03-17,2010-03-17,0.0,0,2,0,120,1080.0
21753,7597,"Boku, Otaryman.",https://cdn.myanimelist.net/images/anime/13/19...,"Boku, Otary-man (otaku + salaryman) is a autob...",No background information,2021-02-16 18:18:17,None,https://myanimelist.net/anime/7597/Boku_Otarym...,https://myanimelist.net/anime/7597/Boku_Otarym...,None,ぼく、オタリーマン。,OVA,1,Finished Airing,"Jan 29, 2010",None,None,[Liverpool],None,"[Panda Factory, Studio PuYUKAI]",Manga,"[Comedy, Seinen, Slice of Life]",35 min.,PG-13 - Teens 13 or older,6.16,2110,7514,7122,3945,4,None,NaN,2010-01-29,2010-01-29,0.0,0,35,0,2100,2100.0
21754,7598,Loups=Garous,https://cdn.myanimelist.net/images/anime/13/26...,In a future governed through the lens of a cam...,No background information,2021-02-16 18:18:19,None,https://myanimelist.net/anime/7598/Loups_Garou...,https://myanimelist.net/anime/7598/Loups_Garou...,"Loups-Garous, Loups=Garous Kihi Subeki Ookami",ルー=ガルー 忌避すべき狼,Movie,1,Finished Airing,"Aug 28, 2010",None,None,[Trans Arts],[Sentai Filmworks],[Production I.G],Novel,"[Sci-Fi, Mystery, Thriller]",1 hr. 38 min.,R - 17+ (violence & profanity),6.30,7030,6893,4031,16107,9,None,NaN,2010-08-28,2010-08-28,0.0,1,38,0,5880,5880.0
